### Assignment : Week 2
## Finding best policies in simple MDPs

Great work making the MDPs in Week 1!

In this assignment, we'll use the simplest RL techniques - Policy and Value iteration to find the best policies (which maximize the discounted total reward) in our MDPs from last week.

Feel free to use your own MDPs, or import them from the OpenAI Gym library.

You can start this assignment during/after reading Grokking Ch-3.

Let us recall the equation to find the value function of agent's states under a policy $\pi$ -
$$v_{\pi}(s) = \sum _{a} \pi(a|s) ~ \left( ~ \sum _{s', r} ~ p(s', r | s, a) ~ \left[r + \gamma v_{\pi}(s') \right] ~ \right)$$

We can observe that the value function $v_{\pi}$ has a lot of circular dependencies on different states. 

To solve such equations, one of the ways is to iteratively calculate the RHS and replace the LHS by it until the $v_{\pi}(s)$ values start to converge. 

The point of convergence makes all the equations simultaneously true and hence is the required solution.

Let us calculate the value functions for some policies in the MDPs we created last week.

## Environment 0 - Bandit Walk

Again, we consider the BW environment on Page 39.

Let's consider what seems to be the most natural policy - always go Right.

This environment is so simple, that we can simply calculate the value functions by hand.

Note that by convention for the terminal states, 
$$v_{\pi}(0) = v_{\pi}(2) = 0$$

Now, 
$$v_{\pi}(1) = 1 + \gamma \cdot v_{\pi}(2) = 1$$

Note both the summations just have one term due to the deterministic nature of the environment and the policy (check which summation was corresponding to which stochastic variable)

## Environment 1 - Slippery Walk

Let's now try to solve the SWF environment from Page 67 for the naturally adversarial policy - always go Left.

Since we have 5 coupled equations for states 1-5 with 5 unknown variables, we'll use Python to bruteforce the solution.

To align with Grokking, let us consider an unusual $\gamma = 1$.

In [1]:
# Step 0 is to import stuff
import gym, gym_walk
import numpy as np
from gym.envs.toy_text.frozen_lake import generate_random_map

In [2]:
# Step 1 is to get the MDP

env = gym.make('SlipperyWalkFive-v0')
swf_mdp = env.P
# swf_mdp

# Note that in Gym, action "Left" is "0" and "Right" is "1"

In [70]:
swf_mdp

{0: {0: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 0, 0.0, True),
   (0.16666666666666666, 0, 0.0, True)],
  1: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 0, 0.0, True),
   (0.16666666666666666, 0, 0.0, True)]},
 1: {0: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 1, 0.0, False),
   (0.16666666666666666, 2, 0.0, False)],
  1: [(0.5000000000000001, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.16666666666666666, 0, 0.0, True)]},
 2: {0: [(0.5000000000000001, 1, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.16666666666666666, 3, 0.0, False)],
  1: [(0.5000000000000001, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.16666666666666666, 1, 0.0, False)]},
 3: {0: [(0.5000000000000001, 2, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.16666666666666666, 4, 0.0, False)],
  1: [(0.5000000000000001, 4, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.16666666666666666, 2, 0.0, Fa

In [71]:
policy = {
    state: {action: 1/len(actions) for action in actions}
    for state, actions in swf_mdp.items()
}

In [72]:
values = {
    state: 0
    for state in swf_mdp
}

In [5]:
def get_new_value_fn(val, mdp, pi, gamma = 1.0):
    epsilon = 1e-10
    delta = 10
    count = 0
    while delta > epsilon:
        count += 1
        delta = 0
        for state in mdp:
            prev = val[state]
            val[state] = sum([prob*sum([tup[0]*(tup[2] + gamma*val[tup[1]]) for tup in mdp[state][action]]) for action, prob in pi[state].items()])
            delta = max(delta, abs(val[state] - prev))
    return val, count

In [6]:
def policy_improvement(val, mdp, gamma=1.0):
    new_pi = dict()
    q = {
        state: {action: 0 for action in mdp[state]}
        for state in mdp
    }
    epsilon = 1e-10
    delta = 10
    count = 0
    while delta > epsilon:
        count += 1
        delta = 0
        for state in mdp:
            for action in mdp[state]:
                prev = q[state][action]
                q[state][action] = sum([tup[0]*(tup[2] + gamma*val[tup[1]]) for tup in mdp[state][action]])
                delta = max(delta, abs(q[state][action] - prev))
    epsilon = 0.1
    for state, results in mdp.items():
        if len(results) == 1:
            new_pi[state] = list(results.keys())[0]
        else:
            a = np.argmax(list(q[state].values()))
            new_pi[state] = {
                action: 1 - epsilon if action == a
                else epsilon/(len(results) - 1)
                for action in results
            }
    return new_pi, q

In [7]:
def value_iteration(mdp, gamma=1.0, epsilon=1e-10):
    values = {
        state: 0
        for state in mdp
    }
    policy = {}
    new_policy = {
        state: {action: 1/len(actions) for action in actions}
        for state, actions in mdp.items()
    }
    count = 0
    while policy != new_policy or not count:
        policy = {
                state: {
                    action: prob for action, prob in results.items()
                }
                for state, results in new_policy.items()
            }
        count += 1
        values, count1 = get_new_value_fn(values, mdp, policy)
        new_policy, q = policy_improvement(values, mdp)
    policy = {
        state: np.argmax(list(actions.values()))
        for state, actions in policy.items()
    } 
    return policy, values, count

In [132]:
p1, val1, c1 = value_iteration(swf_mdp)

In [133]:
p1

{0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 0}

In [134]:
env2 = gym.make('FrozenLake-v1',desc=generate_random_map(size=4))
mdp2 = env2.P
mdp2

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},


In [ ]:
# 0 = LEFT
# 1 = DOWN
# 2 = RIGHT
# 3 = UP

In [135]:
p2, val2, c2 = value_iteration(mdp2)

In [136]:
p2

{0: 3,
 1: 0,
 2: 3,
 3: 3,
 4: 3,
 5: 0,
 6: 3,
 7: 3,
 8: 3,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 2,
 14: 1,
 15: 0}

In [137]:
val2

{0: 0.518033923101253,
 1: 0.5181080293469412,
 2: 0.5114995795659087,
 3: 0.5077445157497942,
 4: 0.5173175655549396,
 5: 0.518865768511948,
 6: 0.4839168505616457,
 7: 0.47144556754802247,
 8: 0.4954268157612993,
 9: 0.5247871160116154,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.6143570861473799,
 14: 0.7674812376675572,
 15: 0.0}

In [3]:
new_frozen_lake_mdp = gym.make('FrozenLake-v1').env.P

In [4]:
new_frozen_lake_mdp

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [8]:
p3, val3, count3 = value_iteration(new_frozen_lake_mdp)

In [9]:
p3

{0: 0,
 1: 3,
 2: 0,
 3: 3,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 3,
 9: 1,
 10: 0,
 11: 0,
 12: 0,
 13: 2,
 14: 1,
 15: 0}

In [10]:
val3

{0: 0.3445172942371296,
 1: 0.28906934503442,
 2: 0.2635251213099903,
 3: 0.23881964108437245,
 4: 0.3502532892582283,
 5: 0.0,
 6: 0.24053663700501518,
 7: 0.0,
 8: 0.39222238330156667,
 9: 0.4747662067433164,
 10: 0.48296789010899044,
 11: 0.0,
 12: 0.0,
 13: 0.5982220923928344,
 14: 0.7835630221202965,
 15: 0.0}